In [1]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import traceback
import base64
from bs4 import BeautifulSoup
import config
import functions
import re

In [2]:
def connection():
    mail_pass = config.mail_pass
    username = config.username
    imap_server = "imap.mail.ru"
    imap = imaplib.IMAP4_SSL(imap_server)
    sts, res = imap.login(username, mail_pass)
    print(res)
    if sts == "OK":
        return imap
    else:
        return False


def encoded_words_to_text(encoded_words):
    # encoded_words=encoded_words.replace("\n", "").replace("\r", "").replace("\t", "").replace(" ", "")
    encoded_word_regex = r"=\?{1}(.+)\?{1}([B|Q])\?{1}(.+)\?{1}="
    charset, encoding, encoded_text = re.match(
        encoded_word_regex, encoded_words
    ).groups()
    if encoding == "B":
        byte_string = base64.b64decode(encoded_text)
    elif encoding == "Q":
        byte_string = quopri.decodestring(encoded_text)
    return byte_string.decode(charset)


def get_attaches(msg):
    attaches = list()
    for part in msg.walk():
        if part.get_content_disposition() == "attachment":
            if "pdf" in part.get_content_type():
                attaches.append(pdf_to_str(part.get_payload(decode=True)))
    if attaches:
        return attaches
    else:
        return [False]


def date_parse(msg_date):
    dt_obj = "".join(str(msg_date[:6]))
    dt_obj = dt_obj.strip("'(),")
    dt_obj = datetime.strptime(dt_obj, "%Y, %m, %d, %H, %M, %S")
    return dt_obj


def pdf_to_str(pdf):
    all_doc = ""
    try:
        with io.BytesIO(pdf) as open_pdf_file:
            read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
            num_pages = read_pdf.getNumPages()
            for page in read_pdf.pages:
                page_content = page
                all_doc += page_content.extractText()
            return all_doc
    except:
        return False

async def send_message(bot_token, message, chat, rpl=None, prv=None):
    bot = Bot(token=bot_token)
    await bot.get_session()
    obj = await bot.send_message(
        chat_id=chat,
        text=message,
        parse_mode="HTML",
        reply_to_message_id=rpl,
        disable_web_page_preview=prv,
    )
    await bot._session.close()
    return obj.message_id

def get_letter_text(body):   
    soup = BeautifulSoup(body, 'html.parser')
    paragraphs = soup.find_all('div')
    text=""
    for paragraph in paragraphs:
        text += paragraph.text + "\n"    
    return text.replace('\xa0', ' ')

def get_text_from_multipart(msg):
    payload = msg.get_payload()[0]
    print(payload["Content-Type"])
    if "text/plain" in payload["Content-Type"]:
        letter_text = (
            (base64.b64decode(payload.get_payload()).decode())
            .lstrip()
            .rstrip()
        )
        letter_text = letter_text.replace("<","").replace(">","")       
    if "text/html" in payload["Content-Type"]:
        letter_text = (
            (base64.b64decode(payload.get_payload()).decode())
            .lstrip()
            .rstrip()
        )
        letter_text=get_letter_text(letter_text)
    if "multipart" in payload["Content-Type"]:
        letter_text = (
            (
                base64.b64decode(
                    payload.get_payload()[0].get_payload()
                ).decode()
            )
            .lstrip()
            .rstrip()
        )
    return letter_text

def post_construct(msg_subj, msg_from, msg_email, letter_text, att_files):
    txt = ""
    txt += (
        "<b>"
        + str(msg_subj)
        + "</b>"
        + "\n\n"
        + str(msg_from)
        + "\n"
        + msg_email
        + "\n"
        + letter_text
        + "\n"
        + "вложения pdf: "
        + str(att_files)
    )
    return txt

In [43]:
imap = connection()

[b'Authentication successful']


In [44]:
status, messages = imap.select("INBOX")  # папка входящие
res, unseen_msg = imap.uid('search', "UNSEEN", "ALL")
unseen_msg = unseen_msg[0].decode("utf-8").split(" ")

In [45]:
if unseen_msg[0]:
    print(unseen_msg)
else:
    print('there are no unseen messages')

['10']


In [46]:
for letter in unseen_msg:
    print(letter)
    attachments=[]
    res, msg = imap.uid('fetch', letter, '(RFC822)')
    msg = email.message_from_bytes(msg[0][1]) # получаем объект email
    if msg.is_multipart():
        print('multipart')                          
    else:
        print('single')                

10
multipart


In [429]:
letter=str(24)
res, msg = imap.uid('fetch', letter, '(RFC822)')
msg = email.message_from_bytes(msg[0][1]) # получаем объект email

In [47]:
from email.policy import default as default_policy
res, msg = imap.uid('fetch', letter, '(RFC822)')
msg = email.message_from_bytes(msg[0][1], policy=default_policy)
body=msg.get_body('plain')
for row in body:
    print(row)

Content-Type
Content-Transfer-Encoding


In [48]:
res, msg_0 = imap.uid('fetch', letter, '(RFC822)')
msg_0[0][0] # Что это за цифры? 15814
b'19 (UID 28 RFC822 {110401}'

b'19 (UID 28 RFC822 {110401}'

In [49]:
#multipart
payload = msg.get_payload() #список, состоит из нескольких частей

In [50]:
payload

In [51]:
def get_text_from_multipart(payload):
    #payload = msg.get_payload()    
    print(payload["Content-Type"])
    if payload.is_multipart():
        print('m')
        letter_text = (
            (
                base64.b64decode(
                    payload.get_payload()[0].get_payload()
                ).decode()
            )
            .lstrip()
            .rstrip()
        )
        
    elif "text/plain" in payload["Content-Type"]:
        print('i')
        letter_text = (
            (base64.b64decode(payload.get_payload()).decode())
            .lstrip()
            .rstrip()
        )
        letter_text = letter_text.replace("<","").replace(">","")
        
    
    elif "text/html" in payload["Content-Type"]:
        print('ii')
        letter_text = (
            (base64.b64decode(payload.get_payload()).decode())
            .lstrip()
            .rstrip()
        )        
        letter_text=get_letter_text(letter_text)                     
    return letter_text

In [31]:
def encode_att_names(str_pl):
    enode_name=re.findall('\=\?.*?\?\=', str_pl)        
    if len(enode_name) == 1:            
        decode_name=(decode_header(enode_name[0])[0][0])            
        decode_name=decode_name.decode("utf-8")            
        str_pl=str_pl.replace(enode_name[0], decode_name)
    if len(enode_name) > 1:             
        nm=''
        for part in enode_name:                
            decode_name=(decode_header(part)[0][0])                        
            decode_name=decode_name.decode("utf-8")
            nm+=decode_name            
        str_pl=str_pl.replace(enode_name[0], nm)    
        for c, i in enumerate(enode_name):           
            if c > 0:                
                str_pl=str_pl.replace(enode_name[c], "").replace("\"",'').rstrip()
    return str_pl  

In [54]:
attachments=list()  
for pl in payload:
    
    print(pl.is_multipart())
    print(pl.get_content_disposition(), pl.get_content_subtype())
    print(pl["Content-Type"])
    
    if pl.is_multipart():                
        next_level=pl.get_payload()
        types=[]
        for level in next_level:
            print(type(level["Content-Type"]))            
            if len(types) == 0: #чтобы два раза не собиралась письмо                
                print(get_text_from_multipart(level))
            hl=level["Content-Type"]
            types.append(hl)                  
    if 'name' in pl["Content-Type"] and pl.get_content_disposition() == 'attachment':               
        str_pl=pl["Content-Type"]               
        str_pl=encode_att_names(str_pl)
        attachments.append(str_pl)
        print(str_pl)
    else:
        pass
        print(pl["Content-Type"])

True
None alternative
multipart/alternative; boundary="--ALT--e8Ba165B7Df82f9b309b135Ba6F5eEb81662683293"
<class 'email.headerregistry._ContentTypeHeader'>
text/plain; charset="utf-8"
i
Ещё одна проверка 
 
С уважением,
Александр Третьяков,

+7 917 222 86 02
<class 'email.headerregistry._ContentTypeHeader'>
multipart/alternative; boundary="--ALT--e8Ba165B7Df82f9b309b135Ba6F5eEb81662683293"
False
attachment pdf
application/pdf; name="Бланк согласие на грипп.pdf"
application/pdf; name="Бланк согласие на грипп.pdf"


In [57]:
attachments

['application/pdf; name="Бланк согласие на грипп.pdf"']

In [56]:
def get_attachments():
    payload = msg.get_payload()
    attachments=list()  
    for pl in payload:
        if 'name' in pl["Content-Type"] and pl.get_content_disposition() == 'attachment':               
            str_pl=pl["Content-Type"]               
            str_pl=encode_att_names(str_pl)
            attachments.append(str_pl)
    return attachments

In [59]:
att_txt=""
for atts in attachments:
    att_txt+=(atts.strip()+"\n")

    

In [60]:
att_txt

'application/pdf; name="Бланк согласие на грипп.pdf"\n'